In [ ]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

In [ ]:
print(tf.__version__)
print(keras.__version__)
print(np.__version__)

2.18.0
3.8.0
2.0.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/Valentini/denoising_dataset.npz" "/content/"


In [ ]:

# Load and split the dataset
data = np.load("/content/denoising_dataset.npz")
noisy = data["noisy"]
clean = data["clean"]

# Split arrays for training and validation
noisy_train, noisy_val, clean_train, clean_val = train_test_split(
    noisy, clean, test_size=0.1, random_state=42
)

# Generators for training and validation data
def train_generator():
    for x, y in zip(noisy_train, clean_train):
        yield x, y

def val_generator():
    for x, y in zip(noisy_val, clean_val):
        yield x, y

# TensorSpec
output_signature = (
    tf.TensorSpec(shape=(128, 128, 1), dtype=tf.float32),
    tf.TensorSpec(shape=(128, 128, 1), dtype=tf.float32)
)

# Create tf.data.Dataset objects
train_dataset = tf.data.Dataset.from_generator(
    train_generator,
    output_signature=output_signature
).shuffle(1000).batch(16).repeat().prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_generator(
    val_generator,
    output_signature=output_signature
).batch(16).prefetch(tf.data.AUTOTUNE)


# Arquitectura de la red U-Net


In [ ]:
def build_unet_transposed(input_shape=(128, 128, 1)):
    inputs = keras.Input(shape=input_shape)

    # Encoder
    conv1 = layers.Conv2D(16, (3, 3), activation="relu", padding="same")(inputs)
    conv1 = layers.Conv2D(16, (3, 3), activation="relu", padding="same")(conv1)
    pool1 = layers.MaxPooling2D((2, 2))(conv1)

    conv2 = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(pool1)
    conv2 = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(conv2)
    pool2 = layers.MaxPooling2D((2, 2))(conv2)

    conv3 = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(pool2)
    conv3 = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(conv3)
    pool3 = layers.MaxPooling2D((2, 2))(conv3)

    # Bottleneck
    conv4 = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(pool3)
    conv4 = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(conv4)

    # Decoder with Conv2DTranspose
    up1 = layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding="same")(conv4)
    merge1 = layers.Concatenate()([up1, conv3])
    conv5 = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(merge1)
    conv5 = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(conv5)

    up2 = layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding="same")(conv5)
    merge2 = layers.Concatenate()([up2, conv2])
    conv6 = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(merge2)
    conv6 = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(conv6)

    up3 = layers.Conv2DTranspose(16, (3, 3), strides=(2, 2), padding="same")(conv6)
    merge3 = layers.Concatenate()([up3, conv1])
    conv7 = layers.Conv2D(16, (3, 3), activation="relu", padding="same")(merge3)
    conv7 = layers.Conv2D(16, (3, 3), activation="relu", padding="same")(conv7)


    # Output layer
    outputs = layers.Conv2D(1, 1, activation="linear")(conv7)


    model = keras.Model(inputs, outputs)
    return model

# Creation of model
model = build_unet_transposed()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 128,  │        160 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 128, 128,  │      2,320 │ conv2d[0][0]      │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 64, 64,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 64, 64,    │      4,640 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 64, 64,    │      9,248 │ conv2d_2[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 32, 32,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 32, 32,    │     18,496 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 32, 32,    │     36,928 │ conv2d_4[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 16, 16,    │          0 │ conv2d_5[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 16, 16,    │     73,856 │ max_pooling2d_2[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 16, 16,    │    147,584 │ conv2d_6[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 32, 32,    │     73,792 │ conv2d_7[0][0]    │
│ (Conv2DTranspose)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 32, 32,    │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 128)              │            │ conv2d_5[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 32, 32,    │     73,792 │ concatenate[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 32, 32,    │     36,928 │ conv2d_8[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_1  │ (None, 64, 64,    │     18,464 │ conv2d_9[0][0]  

 Total params: 535,505 (2.04 MB)

 Trainable params: 535,505 (2.04 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Keras callbacks to fine-tune model training

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint_cb = ModelCheckpoint(
    "best_model.h5",
    save_best_only=True,
    monitor="val_loss",
    mode="min",
    verbose=1
)

early_stopping_cb = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True,
    verbose=1
)

lr_scheduler_cb = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)


In [ ]:
steps_per_epoch = len(noisy_train) // 16
validation_steps = len(noisy_val) // 16

In [ ]:

model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Train model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[checkpoint_cb, early_stopping_cb]
)

In [ ]:
# Save model to be utilized in separate environments
model.save("denoising_model.keras")